# Load embeddings 

In [ ]:
from transformers import AutoModel
import sys
sys.path.append('/scratch/dengm/distent/Disentangle-features/archive')
from features import *
from metrics import *
import os
import torch as t
import torch.nn as nn
import pickle
import numpy as np
import torch as t
def get_acti(samples, emb, tokenizer):
    idList = np.random.choice(emb.shape[0], samples, replace = False)
    words = tokenizer.convert_ids_to_tokens(idList)
    embs = emb[idList]
    return embs, words
# load gpt3
from transformers import GPT2LMHeadModel, GPT2Tokenizer
model_name = "gpt2"
load = True
if not load:
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    emb = model.transformer.wte.weight.detach().numpy()
    embs, words = get_acti(emb.shape[0], emb, tokenizer)
    np.save(f"gpt_embs.npy", embs)
    np.save(f"gpt_words.npy", words)
else:
    embs = np.load(f"gpt_embs.npy")
    words = np.load(f"gpt_words.npy")
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,6,7"
device = t.device("cuda" if t.cuda.is_available() else "cpu")


print(embs.shape, words.shape)



/data/scratch/dengm/anaconda3/envs/edm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(50257, 768) (50257,)


# Solve for features

In [4]:
torch.manual_seed(0)

In [ ]:
f1, e1, inf1 = load_wrapper("gd")

In [5]:
f1, e1, inf1 = wrapper('gd', 'gpt_task', t.Tensor(embs).to(device), init_lamb = 0.62, lamb_left = 0.4, lamb_right = 0.6, guess_factor=8, lr = 5e-3, steps = 8000)

torch.Size([50257, 768])


100%|██████████| 8000/8000 [17:04<00:00,  7.81it/s, accuracy=0, loss=4.04e+5]  


torch.Size([50257, 768])


100%|██████████| 8000/8000 [17:04<00:00,  7.81it/s, accuracy=0, loss=4.49e+5]  


In [7]:
print(inf1, metric_total_acti(f1))

def norminfty(x):
    return x.abs().max(dim = 1)[0] + 1e-8
def normp(p):
    return lambda x: x.abs().pow(p).sum(dim = 1)
def norm_ord(x, p):
    return normp(p)(x)
def sparse_metric_scaled(pw):
    return lambda feat: (norm_ord(feat, pw) / norminfty(feat) ** pw).mean() * (1 + pw) - pw

def sparse_metric_mean_scaled(pw):
    return lambda feat: (norm_ord(feat, pw).mean() / norminfty(feat).mean() ** pw) * (1 + pw) - pw

def sparse_metric_mean(pw):
    return lambda feat: (norm_ord(feat, pw).mean() / norminfty(feat).mean() ** pw)
print(sparse_metric_mean_scaled(1)(f1))
print(sparse_metric_mean_scaled(0.7)(f1))
print(sparse_metric_mean(1)(f1))
print(sparse_metric_mean(0.7)(f1))
# print(inf1, metric_total_acti(f1))

{'loss': 448521.0, 'loss_est': 6.540206901116423, 'lamb': 0.8259901404380798, 'maxA': 1.6520374, 'guess_factor': 8, 'negative_penalty': 1, 'lr': 0.005, 'steps': 8000, 'size': 50257} 2.2875812
tensor(3.5752)
tensor(12.1481)
tensor(2.2876)
tensor(7.5577)


In [11]:

sum_feat = f1.sum(dim = 0)
best_dims = sum_feat.argsort(descending = True)
for i in range(0, 6000, 100):
    print(i, sum_feat[best_dims[i]].item())
    print(get_feat_top(words, f1, best_dims[i], (f1[:, best_dims[i]] > 0.05).sum())[0])
    print()

0 301.19427490234375
['Ġachieves', 'Ġestablishes', 'Ġinvests', 'Ġgenerates', 'Ġdevelops', 'Ġdelivers', 'Ġintegrates', 'Ġcommunicates', 'Ġoperates', 'Ġexpands', 'Ġeliminates', 'Ġregulates', 'Ġcollects', 'Ġprepares', 'Ġenhances', 'Ġremoves', 'Ġadjusts', 'Ġimproves', 'Ġincorporates', 'Ġavoids', 'Ġcontributes', 'Ġdestroys', 'Ġevolves', 'Ġimposes', 'Ġstimulates', 'Ġreplaces', 'Ġearns', 'Ġevaluates', 'Ġdetermines', 'Ġinteracts', 'Ġpublishes', 'Ġoccupies', 'Ġbehaves', 'Ġdominates', 'Ġgoverns', 'Ġreduces', 'Ġperforms', 'Ġrecovers', 'Ġinduces', 'Ġsatisfies', 'Ġconsumes', 'Ġdistinguishes', 'Ġcompletes', 'Ġengages', 'Ġstrives', 'Ġconnects', 'Ġparticipates', 'Ġpromotes', 'Ġprotects', 'Ġinspires', 'Ġsucceeds', 'Ġrestricts', 'Ġstrengthens', 'Ġensures', 'Ġintroduces', 'Ġemploys', 'Ġactivates', 'Ġenables', 'Ġmanages', 'Ġsolves', 'Ġthreatens', 'Ġexecutes', 'Ġfacilitates', 'Ġreinforces', 'Ġattracts', 'Ġreacts', 'Ġundermines', 'Ġsettles', 'Ġinhibits', 'Ġemits', 'Ġutilizes', 'Ġproduces', 'Ġprevents', 'Ġch

In [13]:

sum_feat = t.tensor(embs).sum(dim = 0)
best_dims = sum_feat.argsort(descending = True)
for i in range(0, 128, 1):
    print(i, sum_feat[best_dims[i]].item())
    print(get_feat_top(words, t.tensor(embs), best_dims[i], 20)[0])
    print()

0 14980.1416015625
['DragonMagazine', 'è¦ļéĨĴ', 'EStream', 'ciating', 'ÃĥÃĤÃĥÃĤ', '»Ĵ', 'ĸļ', 'uliffe', 'é¾įå¥ĳå£«', 'Ě', 'Č', 'þ', 'ý', 'Đ', 'Ć', 'Ę', 'ãĥ¯ãĥ³', 'cloneembedreportprint', 'ù', 'Ĉ']

1 14700.4140625
['natureconservancy', 'âĹ¼', 'BuyableInstoreAndOnline', '©¶æ', 'anooga', 'soDeliveryDate', 'ĠCanaver', 'DragonMagazine', 'Downloadha', 'dayName', 'ertodd', 'uyomi', 'iHUD', 'ahime', 'TPPStreamerBot', 'isSpecialOrderable', 'arnaev', 'ļéĨĴ', 'ONSORED', 'jriwal']

2 14644.1962890625
['BuyableInstoreAndOnline', 'Downloadha', 'racuse', 'ertodd', 'arnaev', 'natureconservancy', 'DragonMagazine', 'iHUD', 'âĹ¼', 'soType', 'nesota', 'ĸļ', 'catentry', 'ĠsrfAttach', 'obyl', 'soDeliveryDate', 'ahime', 'SpaceEngineers', 'VERTISEMENT', 'tyard']

3 14533.748046875
['20439', 'emort', 'ĠCosponsors', 'ĠILCS', 'ovember', 'ĠCanaver', 'uberty', 'Downloadha', 'foreseen', 'izoph', 'catentry', 'atonin', 'oglobin', 'BuyableInstoreAndOnline', 'ĠYanuk', 'Recomm', 'rocal', 'govtrack', 'ĠEntityItem', 'ict

In [23]:

from sklearn.decomposition import PCA
pca = PCA(n_components=embs.shape[1])
pca.fit(embs)
new_coords = pca.transform(embs)
sum_feat = t.tensor(new_coords).sum(dim = 0)
best_dims = sum_feat.argsort(descending = True)
for i in range(0, 128, 1):
    print(i, sum_feat[best_dims[i]].item())
    print(get_feat_top(words, t.Tensor(new_coords).to(device), best_dims[i], 20)[0])
    print()

0 0.005957603454589844
['##－', 'ᅦ', 'տ', '[unused164]', '[unused494]', '[unused326]', '[unused751]', '##主', '[unused899]', '[unused209]', '[unused3]', 'ₗ', '[unused933]', '[unused14]', '[unused179]', '[unused749]', '彳', 'კ', '[unused955]', '[unused559]']

1 0.005356550216674805
['throughout', 'to', 'in', '##nostic', 'anymore', 'toward', 'across', '##dened', 'beside', 'aback', 'towards', 'during', 'behind', 'around', 'beforehand', 'for', 'into', 'against', 'between', '##nted']

2 0.005305767059326172
['a', 'is', 'was', 'an', '"', 'the', 'are', 'its', 'were', 'being', 'another', 'it', 'becomes', 'be', 'those', 'became', 'been', 'this', 'your', 'pont']

3 0.004507541656494141
['they', 'there', 'it', 'though', 'she', 'others', 'besides', 'nonetheless', 'consequently', 'therefore', 'however', 'he', 'powerplant', 'importantly', 'alternatively', 'although', 'meantime', 'snails', 'apart', 'indeed']

4 0.004450321197509766
['began', 'ceased', '##uen', '##week', '##nal', 'begun', '##rift', 'ende

# Visualize the directions of a given token

In [62]:

def visualize_by_features(feat, words, word_id):
    print(words[word_id])
    acts = feat[word_id].argsort(descending = True)
    for j in range(3):
        # display all the activations contribute > 1/20 to the sum
        total_act = feat[word_id].sum()
        print(acts[j].item(), feat[word_id, acts[j]].item(), feat[:, acts[j]].sum().item())
        print(get_feat_top(words, feat, acts[j], 20)[0])

def visualize_nearest(feat, words, word_id):
    # get the nearest 10 words
    diff = feat - feat[word_id]
    dist = diff.pow(2).sum(dim = 1)
    nearest = dist.argsort()
    for i in range(20):
        print(words[nearest[i]], dist[nearest[i]].item())

In [45]:
visualize_by_features(f1, words, list(words).index('peking'))

peking
2147 0.18776926398277283 11.760446548461914
['zhejiang', 'guangdong', 'jiangsu', 'hunan', 'shandong', 'henan', 'fujian', 'sichuan', 'tianjin', 'guangzhou', 'yunnan', 'xinjiang', 'shenzhen', 'nanjing', 'taipei', 'beijing', 'shanghai', 'china', 'peking', 'taiwan']
507 0.13848425447940826 11.693121910095215
['harvard', 'yale', 'rutgers', 'stanford', 'princeton', 'tulane', 'georgetown', 'fordham', 'purdue', 'cornell', 'mcgill', 'emory', 'cambridge', 'tufts', 'oxford', 'baylor', 'loyola', 'concordia', 'syracuse', 'clemson']
3786 0.1294572502374649 13.923301696777344
['bucharest', 'istanbul', 'tbilisi', 'seoul', 'tehran', 'helsinki', 'budapest', 'beijing', 'yerevan', 'oslo', 'zagreb', 'belgrade', 'tallinn', 'brussels', 'jakarta', 'nanjing', 'cairo', 'tokyo', 'gothenburg', 'beirut']


In [63]:
visualize_nearest(f1, words, list(words).index('peking'))

peking 0.0
shanghai 0.05828685313463211
beijing 0.05916265770792961
taiwan 0.06983471661806107
nanjing 0.07084673643112183
shenzhen 0.07876468449831009
guangzhou 0.09129001200199127
yunnan 0.10201356559991837
baylor 0.10239443928003311
tianjin 0.10664130002260208
concordia 0.10816189646720886
berlin 0.10853046923875809
clemson 0.10898559540510178
auburn 0.10986308008432388
hopkins 0.11097830533981323
using 0.11136411875486374
hero 0.11137408018112183
field 0.1115109845995903
china 0.11199554800987244
business 0.11212232708930969


In [58]:
visualize_by_features(f1, words, list(words).index('physics'))

physics
2819 0.16798430681228638 8.360572814941406
['journalism', 'philosophy', 'mathematics', 'literature', 'arts', 'theology', 'economics', 'architecture', 'poetry', 'logic', 'physics', 'medicine', 'engineering', 'art', 'science', 'law', 'divinity', 'astronomy', 'humanities', 'music']
3550 0.1636885702610016 13.366540908813477
['zoology', 'biology', 'biochemistry', 'physiology', 'genetics', 'botany', 'geology', 'sociology', 'anatomy', '##biology', 'pathology', 'anthropology', 'linguistics', 'chemistry', 'ecology', '##physics', 'geography', 'philology', 'dentistry', 'psychology']
3303 0.12513267993927002 4.387062072753906
['experiments', 'experiment', 'particle', 'einstein', 'experimentation', 'vacuum', 'physics', 'relativity', 'physicist', '##utter', 'experimenting', 'simulate', 'analogy', 'theoretical', 'experimental', 'tested', 'invented', 'particles', 'tests', 'diagram']


In [ ]:
visualize_nearest(f1, words, list(words).index('physics'))